In [1]:
import sparknlp
sparknlp.start()

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

import pyspark.sql.functions as F
from pyspark.sql.types import StringType

In [3]:
import sparknlp
from sparknlp.pretrained import PretrainedPipeline
#create or get Spark Session
#spark = sparknlp.start()
sparknlp.version()


'2.5.3'

In [4]:
# Import Spark NLP 
from sparknlp.base import *
from sparknlp.annotator import *
#from sparknlp.embeddings import *
from sparknlp.pretrained import PretrainedPipeline
import sparknlp
# Start Spark Session with Spark NLP
spark = sparknlp.start()
# Download a pre-trained pipeline 
pipeline = PretrainedPipeline('explain_document_dl', lang='en')
# Your testing dataset
text = """""
The Mona Lisa is a 16th century oil painting created by Leonardo. 
It’s held at the Louvre in Paris.
"""
# Annotate your testing dataset
result = pipeline.annotate(text)
# What’s in the pipeline
list(result.keys())


explain_document_dl download started this may take some time.
Approx size to download 168.4 MB
[OK!]


['entities',
 'stem',
 'checked',
 'lemma',
 'document',
 'pos',
 'token',
 'ner',
 'embeddings',
 'sentence']

In [39]:
! wget http://www.gutenberg.org/cache/epub/8387/pg8387.txt #hunger

--2020-08-01 23:30:51--  http://www.gutenberg.org/cache/epub/8387/pg8387.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 382801 (374K) [text/plain]
Saving to: ‘pg8387.txt’

pg8387.txt          100%[===================>] 373.83K   719KB/s    in 0.5s    

2020-08-01 23:30:52 (719 KB/s) - ‘pg8387.txt’ saved [382801/382801]



In [63]:
f = open("pg8387.txt", "r",encoding="utf-8-sig")
ulyses=f.read()
f.close()

In [64]:
spark_df = spark.createDataFrame([[ulyses]]).toDF("text")
#park_df = spark.createDataFrame(ulyses)

In [65]:
spark_df.show()

+--------------------+
|                text|
+--------------------+
|The Project Guten...|
+--------------------+



In [66]:
from sparknlp.base import *

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")\
.setCleanupMode("shrink")

doc_df = documentAssembler.transform(spark_df)

doc_df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                            document|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|The Project Gutenberg EBook of Hunger, by Knut Hamsun

This eBook is for the use of anyone anywhe...|[[document, 0, 373149, The Project Gutenberg EBook of Hunger, by Knut Hamsun This eBook is for th...|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------

In [67]:
doc_df.select('document.result','document.begin','document.end').show(truncate=100)

+----------------------------------------------------------------------------------------------------+-----+--------+
|                                                                                              result|begin|     end|
+----------------------------------------------------------------------------------------------------+-----+--------+
|[The Project Gutenberg EBook of Hunger, by Knut Hamsun This eBook is for the use of anyone anywhe...|  [0]|[373149]|
+----------------------------------------------------------------------------------------------------+-----+--------+



In [68]:
import pyspark.sql.functions as F

doc_df.withColumn(
    "tmp", 
    F.explode("document"))\
    .select("tmp.*")\
    .show(truncate=300)

+-------------+-----+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+----------+
|annotatorType|begin|   end|                                                                                                                                                                                                                                                                                                      result|       metadata|embeddings|
+-------------+-----+------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [69]:
doc_df.select("document.result").take(1)

[Row(result=['The Project Gutenberg EBook of Hunger, by Knut Hamsun This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you\'ll have to check the laws of the country where you are located before using this ebook. Title: Hunger Author: Knut Hamsun Posting Date: October 2, 2014 [EBook #8387] Release Date: June, 2005 First Posted: July 6, 2003 Language: English *** START OF THIS PROJECT GUTENBERG EBOOK HUNGER *** Produced by Eric Eldred, Robert Connal, and the Online Distributed Proofreading Team HUNGER by KNUT HAMSUN Translated from the Norwegian by GEORGE EGERTON _With an introduction by Edwin Bjorkman_ Knut Hamsun Since the death of Ibsen and Strindberg, Hamsun is undoubtedly the foremost creat

In [70]:
from sparknlp.annotator import *

# we feed the document column coming from Document Assembler

sentenceDetector = SentenceDetector().setInputCols('document').setOutputCol('sentences').setExplodeSentences(True)

In [71]:
sent_df = sentenceDetector.transform(doc_df)



In [72]:
from pyspark.sql.functions import col

In [73]:
sent_df.select(sent_df.sentences.result[0]).take(1)

[Row(sentences.result[0]='The Project Gutenberg EBook of Hunger, by Knut Hamsun This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever.')]

In [74]:
cow=sent_df.select(F.explode('sentences.result').alias('text'))#.show(truncate=100)

In [91]:
#ulyses=sent_df.select(sent_df.sentences.result[0]).withColumnRenamed("sentences.result[0]", "text").show()#.select('result').collect()[0][0]

+--------------------+
|                text|
+--------------------+
|The Project Guten...|
|You may copy it, ...|
|[Illustration] Ul...|
|A yellow dressing...|
|He held the bowl ...|
|Halted, he peered...|
|Come up, you fear...|
|Solemnly he came ...|
|He faced about an...|
|Then, catching si...|
|Stephen Dedalus, ...|
|Buck Mulligan pee...|
|  —Back to barracks!|
|    he said sternly.|
|He added in a pre...|
| Slow music, please.|
|Shut your eyes, g...|
|         One moment.|
|A little trouble ...|
|       Silence, all.|
+--------------------+
only showing top 20 rows



In [75]:
cow.show(truncate=100)

+----------------------------------------------------------------------------------------------------+
|                                                                                                text|
+----------------------------------------------------------------------------------------------------+
|The Project Gutenberg EBook of Hunger, by Knut Hamsun This eBook is for the use of anyone anywher...|
|You may copy it, give it away or re-use it under the terms of the Project Gutenberg License inclu...|
|If you are not located in the United States, you'll have to check the laws of the country where y...|
|Title: Hunger Author: Knut Hamsun Posting Date: October 2, 2014 [EBook #8387] Release Date: June,...|
|Those approaching most nearly to his position are probably Selma Lagerlöf in Sweden and Henrik Po...|
|Both these, however, seem to have less than he of that width of outlook, validity of interpretati...|
|           His reputation is not confined to his own country or the two 

In [76]:
from pyspark.sql.types import StringType

In [77]:
sents_df = spark.createDataFrame(sents, StringType()).toDF("text")

NameError: name 'sents' is not defined

In [21]:
sents_df.show(10)

+--------------------+
|                text|
+--------------------+
|The Project Guten...|
|You may copy it, ...|
|[Illustration] Ul...|
|A yellow dressing...|
|He held the bowl ...|
|Halted, he peered...|
|Come up, you fear...|
|Solemnly he came ...|
|He faced about an...|
|Then, catching si...|
+--------------------+
only showing top 10 rows



In [22]:
sents_df = documentAssembler.transform(sents_df)

sents_df.show(truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                            document|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|The Project Gutenberg EBook of Ulysses, by James Joyce This eBook is for the use of anyone anywhe...|[[document, 0, 153, The Project Gutenberg EBook of Ulysses, by James Joyce This eBook is for the ...|
|You may copy it, give it away or re-use it under the terms of the Project Gutenberg License inclu...|[[document, 0, 399, You may copy it, give it away or re-use it under the terms of 

In [23]:
token_df = tokenizer.fit(sents_df).transform(sents_df)


In [24]:


token_df.select('token.result').take(13)

[Row(result=['The', 'Project', 'Gutenberg', 'EBook', 'of', 'Ulysses,', 'by', 'James Joyce', 'This', 'eBook', 'is', 'for', 'the', 'use', 'of', 'anyone', 'anywhere', 'at', 'no', 'cost', 'and', 'with', 'almost', 'no', 'restrictions', 'whatsoever.']),
 Row(result=['You', 'may', 'copy', 'it,', 'give', 'it', 'away', 'or', 're', 'use', 'it', 'under', 'the', 'terms', 'of', 'the', 'Project', 'Gutenberg', 'License', 'included', 'with', 'this', 'eBook', 'or', 'online', 'at', 'www.gutenberg.org', 'Title:', 'Ulysses', 'Author:', 'James Joyce', 'Release', 'Date:', 'August', '1,', '2008', '[EBook', '#4300]', 'Last', 'Updated:', 'December', '27,', '2019', 'Language:', 'English', 'Character', 'set', 'encoding:', 'UTF', '8', '***', 'START', 'OF', 'THIS', 'PROJECT', 'GUTENBERG', 'EBOOK', 'ULYSSES', '***', 'Produced', 'by', 'Col', 'Choat,', 'and', 'David', 'Widger.']),
 Row(result=['[Illustration]', 'Ulysses', 'by', 'James Joyce', 'Contents', '—', 'I', '—', '[', '1', ']', '[', '2', ']', '[', '3', ']', '—'

In [28]:
import spacy
nlp=spacy.load('en_core_web_md')

In [29]:
# Construction from class
from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()

In [31]:
raw_text=(""""He held the bowl aloft and intoned: —_Introibo ad altare Dei_.',
 'Halted, he peered down the dark winding stairs and called out coarsely: —Come up, Kinch!',
 'Come up, you fearful jesuit!'""")

In [35]:
nlp.add_pipe(nlp.create_pipe('sentencizer')) # updated
doc = nlp(ulyses)
sentences = [sent.string.strip() for sent in doc.sents]

ValueError: [E007] 'sentencizer' already exists in pipeline. Existing names: ['tagger', 'parser', 'ner', 'sentencizer']

In [36]:
doc = nlp(ulyses)
sentences = [sent.string.strip() for sent in doc.sents]

ValueError: [E088] Text of length 1531927 exceeds maximum of 1000000. The v2.x parser and NER models require roughly 1GB of temporary memory per 100,000 characters in the input. This means long texts may cause memory allocation errors. If you're not using the parser or NER, it's probably safe to increase the `nlp.max_length` limit. The limit is in number of characters, so you can check whether your inputs are too long by checking `len(text)`.

In [37]:
len(ulyses)

1531927

In [33]:
sentences

['"He held the bowl aloft and intoned: —_Introibo ad altare Dei_.\',',
 "'Halted, he peered down the dark winding stairs and called out coarsely: —Come up, Kinch!'",
 ',',
 "'Come up, you fearful jesuit!'"]

In [99]:

from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")\
.setCleanupMode("shrink")

sentenceDetector = SentenceDetector().\
setInputCols(['document']).\
setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 sentenceDetector,
 tokenizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [26]:
sents[3424]

'And just imagine that.'

In [27]:

spark_df = spark.createDataFrame([[sents]]).toDF('text')

In [28]:
spark_df = spark.read.text("4300-0.txt").toDF('text')

In [41]:
spark_df.select('text').collect()[25]

Row(text='Produced by Col Choat, and David Widger.')

In [100]:
result = pipelineModel.transform(sent_df)

In [103]:
result.show(10)

+--------------------+--------------------+--------------------+--------------------+
|                text|            document|           sentences|               token|
+--------------------+--------------------+--------------------+--------------------+
|
The Project Gute...|[[document, 0, 15...|[[document, 0, 15...|[[token, 0, 2, Th...|
+--------------------+--------------------+--------------------+--------------------+



In [56]:
result.select('sentences.result').take(3)

[Row(result=[]),
 Row(result=['The Project Gutenberg EBook of Ulysses, by James Joyce']),
 Row(result=[])]

In [26]:
token_df.show(10)

+--------------------+--------------------+--------------------+
|                text|            document|               token|
+--------------------+--------------------+--------------------+
|The Project Guten...|[[document, 0, 15...|[[token, 0, 2, Th...|
|You may copy it, ...|[[document, 0, 39...|[[token, 0, 2, Yo...|
|[Illustration] Ul...|[[document, 0, 31...|[[token, 0, 13, [...|
|A yellow dressing...|[[document, 0, 89...|[[token, 0, 0, A,...|
|He held the bowl ...|[[document, 0, 61...|[[token, 0, 1, He...|
|Halted, he peered...|[[document, 0, 87...|[[token, 0, 6, Ha...|
|Come up, you fear...|[[document, 0, 27...|[[token, 0, 3, Co...|
|Solemnly he came ...|[[document, 0, 54...|[[token, 0, 7, So...|
|He faced about an...|[[document, 0, 99...|[[token, 0, 1, He...|
|Then, catching si...|[[document, 0, 13...|[[token, 0, 4, Th...|
+--------------------+--------------------+--------------------+
only showing top 10 rows



In [57]:
import string
string.punctuation


'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [53]:
import re

In [55]:
text

'""\nThe Mona Lisa is a 16th century oil painting created by Leonardo. \nIt’s held at the Louvre in Paris.\n'

In [64]:
re.sub("[\!\"\#\$\%\&\\\'\(\)\*\+\,\-\.\/\:\;\<\=\>\?\@\[\\\\\]\^\_\`\{\|\}\~\’\n]",'',text)

'The Mona Lisa is a 16th century oil painting created by Leonardo Its held at the Louvre in Paris'

## Dikkat

In [78]:
documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")\
.setCleanupMode("shrink")

sentenceDetector = SentenceDetector().\
    setInputCols(['document']).\
    setOutputCol('sentences')

tokenizer = Tokenizer() \
    .setInputCols(["sentences"]) \
    .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")\
    .setLowercase(False)\
    .setCleanupPatterns(["[^\w\d\s\.\!\?]"])

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)\

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("lemma")

pos = PerceptronModel.pretrained("pos_anc", 'en')\
      .setInputCols("clean_text", "cleanTokens")\
      .setOutputCol("pos")

chunker = Chunker()\
    .setInputCols(["sentences", "pos"])\
    .setOutputCol("chunk")\
    .setRegexParsers(["<DT>+<JJ>*<NN>"])

tokenassembler = TokenAssembler()\
    .setInputCols(["sentences", "cleanTokens"]) \
    .setOutputCol("clean_text")\
 #   .setPreservePosition(True)

tokenizer2 = Tokenizer() \
    .setInputCols(["clean_text"]) \
    .setOutputCol("token2")

glove_embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["clean_text", "lemma"])\
          .setOutputCol("embeddings")


onto_ner = NerDLModel.pretrained("onto_100", 'en') \
          .setInputCols(["clean_text", "token2", "embeddings"]) \
          .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentences", "token", "ner"]) \
  .setOutputCol("ner_chunk")




nlpPipeline = Pipeline(stages=[
     documentAssembler,
     sentenceDetector,
     tokenizer,
     normalizer,
     stopwords_cleaner,
     lemma,
     
     tokenassembler,
     tokenizer2,
     pos,
     chunker,
     glove_embeddings,
     onto_ner,
     ner_converter
    
 ])


empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)



lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]
pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[OK!]
glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[OK!]
onto_100 download started this may take some time.
Approximate size to download 13.5 MB
[OK!]


In [79]:
cow_result = pipelineModel.transform(cow)

cow_result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|                text|            document|           sentences|               token|          normalized|         cleanTokens|               lemma|          clean_text|              token2|                 pos|chunk|          embeddings|                 ner|           ner_chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+--------------------+
|The Project Guten...|[[document, 0, 20...|[[document, 0, 20...|[[token, 0, 2, Th...|[[token, 0, 2, Th...|[[token, 4, 10, P...|[[token, 4, 10, P...|[[documen

In [80]:
cow_result.select('text', 'clean_text.result').show(20,truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                              result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|The Project Gutenberg EBook of Hunger, by Knut Hamsun This eBook is for the use of anyone anywher...|[Project Gutenberg EBook Hunger Knut Hamsun eBook use anyone anywhere United States parts world c...|
|You may copy it, give it away or re-use it under the terms of the Project Gutenberg License inclu...| [may copy give away reuse terms Project Gutenberg License included eBook online w

In [56]:
n=2
result.select('chunk').take(500)#[0]#[0][n][3]

NameError: name 'result' is not defined

In [313]:
n=2
result.select('pos').take(50)[0]#[0][n][3]

Row(pos=[Row(annotatorType='pos', begin=4, end=10, result='NNP', metadata={'word': 'Project'}, embeddings=[]), Row(annotatorType='pos', begin=12, end=20, result='NNP', metadata={'word': 'Gutenberg'}, embeddings=[]), Row(annotatorType='pos', begin=22, end=26, result='NNP', metadata={'word': 'EBook'}, embeddings=[]), Row(annotatorType='pos', begin=31, end=37, result='NNP', metadata={'word': 'Ulysses'}, embeddings=[]), Row(annotatorType='pos', begin=43, end=47, result='NNP', metadata={'word': 'James'}, embeddings=[]), Row(annotatorType='pos', begin=49, end=53, result='NNP', metadata={'word': 'Joyce'}, embeddings=[]), Row(annotatorType='pos', begin=60, end=64, result='NN', metadata={'word': 'eBook'}, embeddings=[]), Row(annotatorType='pos', begin=77, end=79, result='NN', metadata={'word': 'use'}, embeddings=[]), Row(annotatorType='pos', begin=84, end=89, result='NN', metadata={'word': 'anyone'}, embeddings=[]), Row(annotatorType='pos', begin=91, end=98, result='RB', metadata={'word': 'anyw

In [314]:
n=2
result.select('chunk').take(50)[0]#[0][n][3]

Row(chunk=[])

In [81]:

cow_result.withColumn(
    "tmp", 
    F.explode("chunk")) \
    .select("tmp.*").select("begin","end","result","metadata.sentence").show(100,truncate = 100)

+-----+---+--------------------------------+--------+
|begin|end|                          result|sentence|
+-----+---+--------------------------------+--------+
|    9| 20|                    every reason|       0|
|   58| 67|                      every step|       0|
|   72| 82|                     every cross|       0|
|   42| 59|              another expression|       0|
|   10| 23|                  another reason|       0|
|   71| 80|                      every time|       0|
|   40| 50|                     every stone|       0|
|   10| 29|            every possible thing|       0|
|   29| 38|                      every word|       0|
|    0| 23|        Neither did the quantity|       0|
|   30| 44|                 every newspaper|       0|
|   41| 50|                      every week|       0|
|   17| 37|           another, and they fit|       0|
|   29| 38|                      every word|       0|
|  102|113|                    every moment|       0|
|   17| 28|                 

In [82]:
cow_result.select('clean_text.result', 'cleanTokens.result').show(50,truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                              result|                                                                                              result|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|[Project Gutenberg EBook Hunger Knut Hamsun eBook use anyone anywhere United States parts world c...|[Project, Gutenberg, EBook, Hunger, Knut, Hamsun, eBook, use, anyone, anywhere, United, States, p...|
| [may copy give away reuse terms Project Gutenberg License included eBook online www.gutenberg.org.]|[may, copy, give, away, reuse, terms, Project, Gutenberg, License, included, eBook

In [86]:
cow_result.select(F.explode(F.arrays_zip('cleanTokens.result', 'ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ner_label")).show(100,truncate = 100)


+-----------------+-------------+
|            token|    ner_label|
+-----------------+-------------+
|          Project|            O|
|        Gutenberg|B-WORK_OF_ART|
|            EBook|I-WORK_OF_ART|
|           Hunger|            O|
|             Knut|     B-PERSON|
|           Hamsun|     I-PERSON|
|            eBook|            O|
|              use|            O|
|           anyone|            O|
|         anywhere|            O|
|           United|        B-GPE|
|           States|        I-GPE|
|            parts|         null|
|            world|         null|
|             cost|         null|
|           almost|         null|
|     restrictions|         null|
|       whatsoever|         null|
|                .|         null|
|              may|            O|
|             copy|            O|
|             give|            O|
|             away|            O|
|            reuse|            O|
|            terms|            O|
|          Project|        B-ORG|
|        Guten

In [84]:
result_ner=cow_result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata')).alias("cols")) \
.select(F.expr("cols['0']").alias("ner_chunk"),
        F.expr("cols['1']['entity']").alias("ner_label"))

In [85]:
result_ner.show(10)

+--------------------+-----------+
|           ner_chunk|  ner_label|
+--------------------+-----------+
|     Gutenberg EBook|WORK_OF_ART|
|         Knut Hamsun|     PERSON|
|       United States|        GPE|
|Project Gutenberg...|        ORG|
|       United States|        GPE|
|       Hunger Author|WORK_OF_ART|
|         Knut Hamsun|WORK_OF_ART|
|           October 2|       DATE|
|2014 [EBook #8387...|       DATE|
|                June|       DATE|
+--------------------+-----------+
only showing top 10 rows



In [87]:
result_ner.filter(result_ner.ner_label == "PERSON").take(2)

[Row(ner_chunk='Knut Hamsun', ner_label='PERSON'),
 Row(ner_chunk='GEORGE EGERTON', ner_label='PERSON')]

In [184]:
result.take(1)

[Row(text='The Project Gutenberg EBook of Ulysses, by James Joyce This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.', document=[Row(annotatorType='document', begin=0, end=153, result='The Project Gutenberg EBook of Ulysses, by James Joyce This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.', metadata={'sentence': '0'}, embeddings=[])], sentences=[Row(annotatorType='document', begin=0, end=153, result='The Project Gutenberg EBook of Ulysses, by James Joyce This eBook is for the use of anyone anywhere at no cost and with almost no restrictions whatsoever.', metadata={'sentence': '0'}, embeddings=[])], token=[Row(annotatorType='token', begin=0, end=2, result='The', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=4, end=10, result='Project', metadata={'sentence': '0'}, embeddings=[]), Row(annotatorType='token', begin=12, end=20, result='Gutenberg', metadata={'sente

In [88]:
l=result_ner.filter(result_ner.ner_label == "PERSON").select(F.expr("ner_chunk")).collect()

In [89]:
import re
names=list(([re.sub("[^a-zA-Z\s\']","",l_[0]).title() for l_ in l if len(l_[0].split())==2 and "’" not in l_[0]]))

In [90]:
len(set(names))

25

In [91]:
set(names)

{'Barrabas Rosebud',
 'Bernt Akers',
 'Bookseller Pascha',
 'Carl Johann',
 'Edwin Bjrkman',
 'Eric Eldred',
 'George Egerton',
 'God Bless',
 'Gregory B',
 "Han'S Hill",
 'Hans Pauli',
 'Heavenly Father',
 'Herr Tangen',
 'Ivar Kareno',
 'Jens Olaj',
 "Jens Olaj'S",
 'Johannn Arendt',
 'Kierulfjoachim Kierulf',
 'Knut Hamsun',
 "Miss Andersen'S",
 'Selma Lagerlf',
 'Student Pettersen',
 'Tangenandreas Tangen',
 'Von Tangen',
 'Waldemar Atterdag'}

In [92]:
from collections import Counter

In [93]:
counter=Counter(names).most_common(350)   # Spark NLP

In [94]:
count_2plus=[(name,count) for name,count in counter if count>1]

In [95]:
count_2plus

[('Hans Pauli', 4),
 ('Carl Johann', 3),
 ('Jens Olaj', 3),
 ('Knut Hamsun', 2),
 ("Han'S Hill", 2),
 ('Herr Tangen', 2),
 ('Bernt Akers', 2)]

In [ ]:
-Leopold Bloom

- Marion (Molly) Bloom

- Stephen Dedalus

- Malachi (Buck) Mulligan

Haines

- Hugh (“Blazes”) Boylan

Millicent (Milly) Bloom

Simon Dedalus

A.E. (George Russell)

Richard Best

Edy Boardman

- Josie (née Powell) and Denis Breen

Cissy, Jacky, and Tommy Caffrey

The citizen

Martha Clifford

Bella Cohen

-Martin Cunningham

Garrett Deasy

Dilly, Katey, Boody, and Maggy Dedalus

Patrick Dignam, Mrs. Dignam, and Patrick Dignam, Jr.

-Ben Dollard

John Eglinton

Richie, Sara (Sally), and Walter Goulding

Zoe Higgins

Joe Hynes

-Corny Kelleher

Mina Kennedy and Lydia Douce

-Ned Lambert

Lenehan

Lynch

Thomas W. Lyster

Gerty MacDowell

John Henry Menton

Episode Twelve’s Nameless Narrator

City Councillor Nannetti

J. J. O’Molloy

Jack Power

Kitty Ricketts

Florry Talbot

In [161]:
#result=result.filter(~result.text.contains('[')).filter(~result.text.contains(']'))\
     #           .filter(~result.text.rlike('\—\s*I+\s*\—')).filter(~result.text.rlike('\*{3}\s*.*\s*\*{3}'))\

#result=result.filter(result1['text']!='')

In [160]:
#result.select('text').show(20,truncate=False)

In [161]:
#tokenizer = Tokenizer() \
#    .setInputCols(["document"]) \
#    .setOutputCol("token")
#
#
#nlpPipeline = Pipeline(stages=[
#     documentAssembler,
#     tokenizer,
#     normalizer,
#     stopwords_cleaner,
#     tokenassembler
# ])
#
#
#empty_df = spark.createDataFrame([['']]).toDF("text")
#
#pipelineModel = nlpPipeline.fit(empty_df)
#
#result1 = pipelineModel.transform(result)
#
#result1.select('text', 'clean_text.result').show(truncate=False)

In [203]:
!wget https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt

--2020-06-08 20:49:26--  https://raw.githubusercontent.com/mahavivo/vocabulary/master/lemmas/AntBNC_lemmas_ver_001.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.184.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.184.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1348552 (1.3M) [text/plain]
Saving to: ‘AntBNC_lemmas_ver_001.txt’

AntBNC_lemmas_ver_0 100%[===================>]   1.29M  2.45MB/s    in 0.5s    

2020-06-08 20:49:29 (2.45 MB/s) - ‘AntBNC_lemmas_ver_001.txt’ saved [1348552/1348552]



In [162]:
lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")

In [163]:

documentAssembler = DocumentAssembler()\
.setInputCol("text")\
.setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [209]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [164]:

result = pipelineModel.transform(ulyses)

result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|The Project Guten...|[[document, 0, 15...|[[token, 0, 2, Th...|[[token, 0, 2, th...|[[token, 0, 2, Th...|
|You may copy it, ...|[[document, 0, 39...|[[token, 0, 2, Yo...|[[token, 0, 2, yo...|[[token, 0, 2, Yo...|
|[Illustration] Ul...|[[document, 0, 31...|[[token, 0, 13, [...|[[token, 0, 13, [...|[[token, 0, 13, [...|
|A yellow dressing...|[[document, 0, 89...|[[token, 0, 0, A,...|[[token, 0, 0, a,...|[[token, 0, 0, A,...|
|He held the bowl ...|[[document, 0, 61...|[[token, 0, 1, He...|[[token, 0, 1, he...|[[token, 0, 1, He...|
|Halted, he peered...|[[document, 0, 87...|[[token, 0, 5, Ha...|[[token, 0, 5, ha...|[[token, 0, 5, Ha...|
|Come up, you fear...|[[document, 0, 

In [166]:

result.select('lemma.result').show(truncate=80)

+--------------------------------------------------------------------------------+
|                                                                          result|
+--------------------------------------------------------------------------------+
|[The, Project, Gutenberg, EBook, of, Ulysses, ,, by, James, Joyce, This, eBoo...|
|[You, may, copy, it, ,, give, it, away, or, re-use, it, under, the, term, of,...|
|[[Illustration], Ulysses, by, James, Joyce, Contents, —, I, —, [, 1, ], [, 2,...|
|[A, yellow, dressinggown, ,, ungirdled, ,, be, sustain, gently, behind, he, o...|
|   [He, hold, the, bowl, aloft, and, intone, :, —_Introibo, ad, altare, Dei_, .]|
|[Halted, ,, he, peer, down, the, dark, wind, stairs, and, call, out, coarsely...|
|                                          [Come, up, ,, you, fearful, jesuit, !]|
|               [Solemnly, he, come, forward, and, mount, the, round, gunrest, .]|
|[He, face, about, and, bless, gravely, thrice, the, tower, ,, the, surround, ...|
|[Th

In [167]:

result_df = result.select(F.explode(F.arrays_zip('token.result', 'stem.result',  'lemma.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("stem"),
        F.expr("cols['2']").alias("lemma")).toPandas()

result_df.head(10)

,token,stem,lemma
0,The,the,The
1,Project,project,Project
2,Gutenberg,gutenberg,Gutenberg
3,EBook,ebook,EBook
4,of,of,of
5,Ulysses,ulyss,Ulysses
6,",",",",","
7,by,by,by
8,James,jame,James
9,Joyce,joyc,Joyce


In [176]:
ngrams_cum = NGramGenerator() \
            .setInputCols(["token"]) \
            .setOutputCol("ngrams") \
            .setN(5) \
            .setEnableCumulative(False)\
            .setDelimiter("+") # Default is space
    
# .setN(3) means, take bigrams and trigrams.

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 ngrams_cum
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

result = pipelineModel.transform(ulyses)

result.select('ngrams.result').show(truncate=200)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|                                                                                                                                                                                                  result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|[The+Project+Gutenberg+EBook+of, Project+Gutenberg+EBook+of+Ulysses, Gutenberg+EBook+of+Ulysses+,, EBook+of+Ulysses+,+by, of+Ulysses+,+by+James, Ulysses+,+by+James+Joyce, ,+by+James+Joyce+This, by+...|
|[You+may+copy+it+,, may+copy+it+,+give, copy+it+,+give+it, it+,+give+it+away, ,+give+it+away+or, give+it+away+or+re-use, it+away+or+re-use+it, away+or+re-use+it+under, or+re-use+it+under+

In [169]:
! wget 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv

--2020-07-26 14:55:46--  https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed-sample.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.104.213
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.104.213|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10484510 (10.0M) [text/csv]
Saving to: ‘pubmed-sample.csv.1’

pubmed-sample.csv.1 100%[===================>]  10.00M  12.5MB/s    in 0.8s    

2020-07-26 14:55:47 (12.5 MB/s) - ‘pubmed-sample.csv.1’ saved [10484510/10484510]



In [170]:

import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("./pubmed-sample.csv")\
                .filter("AB IS NOT null")\
                .withColumnRenamed("AB", "text")\
                .drop("TI")

pubMedDF.show(truncate=50)

+--------------------------------------------------+
|                                              text|
+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
|BACKGROUND: At present, it is one of the most i...|
|OBJECTIVE: To investigate the relationship betw...|
|Combined EEG/fMRI recording has been used to lo...|
|Kohlschutter syndrome is a rare neurodegenerati...|
|Statistical analysis of neuroimages is commonly...|
|The synthetic DOX-LNA conjugate was characteriz...|
|Our objective was to compare three different me...|
|We conducted a phase II study to assess the eff...|
|"Monomeric sarcosine oxidase (MSOX) is a flavoe...|
|We presented the tachinid fly Exorista japonica...|
|The literature dealing with the water conductin...|
|A novel approach to synthesize chitosan-O-isopr...|
|An HPLC-ESI-MS-MS method has been developed for...|
|The localizing and lateralizing values of eye a...|
|OBJECTIVE: To evaluate the effectiveness and 

In [179]:
entities = ['wake', 'sleep', 'turn','take','ulyses','bake','shave']
with open ('ulyses_entities.txt', 'w') as f:
    for i in entities:
        f.write(i+'\n')

In [180]:
entity_extractor = TextMatcher() \
    .setInputCols(["document",'token'])\
    .setOutputCol("matched_entities")\
    .setEntities("ulyses_entities.txt")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 entity_extractor
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [181]:
result = pipelineModel.transform(ulyses)

In [182]:
result.select('matched_entities.result', 'matched_entities.begin',  'matched_entities.end').collect()

[Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begin=[], end=[]),
 Row(result=[], begi

In [184]:
result_df = result.select(F.explode(F.arrays_zip('matched_entities.result', 'matched_entities.begin',  'matched_entities.end')).alias("cols1")) \
.select(F.expr("cols1['0']").alias("matched_entities"),
        F.expr("cols1['1']").alias("begin"),
        F.expr("cols1['2']").alias("end"))#.toPandas()

result_df.show(50)

+----------------+-----+---+
|matched_entities|begin|end|
+----------------+-----+---+
|           shave|   21| 25|
|            turn|   97|100|
|            turn|   12| 15|
|            wake|    7| 10|
|            turn|  115|118|
|            take|   18| 21|
|           sleep|   11| 15|
|            take|   27| 30|
|            take|    5|  8|
|           sleep|   11| 15|
|            wake|   36| 39|
|           sleep|    3|  7|
|            take|   25| 28|
|           sleep|   67| 71|
|            turn|   12| 15|
|            take|   10| 13|
|            turn|   10| 13|
|            take|    6|  9|
|            turn|   19| 22|
|            turn|   84| 87|
|            take|   17| 20|
|            turn|   41| 44|
|            wake|   46| 49|
|            take|   18| 21|
|            take|   36| 39|
|            take|    6|  9|
|            take|   25| 28|
|            turn|   32| 35|
|            take|   32| 35|
|            wake|   44| 47|
|            turn|   16| 19|
|            w

In [243]:
result.select('matched_entities.result').show()

+------+
|result|
+------+
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
|    []|
+------+
only showing top 20 rows



In [287]:
rules = '''
Stephen\s(?!Proto|Cardinal)[A-Z]\w+, followed by "Stephen"
Simon\s[A-Z]\w+, followed by "Simon"
'''

with open('ulyses_regex_rules.txt', 'w') as f:
    
    f.write(rules)

In [98]:


rules = '''
renal\s\w+, started with 'renal'
cardiac\s\w+, started with 'cardiac'
\b\w+ly\b, ending with 'ly'
\b[A-Z]\w+ly\b, staring with a capital letter ending with 'ly'
\S*\d+\S*, match any word that contains numbers
(\d+).?(\d{3})\s*(mg|ml|g), match medication metrics
'''

with open('ulyses_regex_rules.txt', 'w') as f:
    
    f.write(rules)
    
f.close()

In [413]:
result.columns

['text', 'document', 'token', 'matched_entities']

In [240]:
ulyses_result.select('text','clean_text').show(100,truncate=100)

+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|                                                                                                text|                                                                                          clean_text|
+----------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------+
|The Project Gutenberg EBook of Ulysses, by James Joyce This eBook is for the use of anyone anywhe...|[[document, 0, 105, Project Gutenberg EBook Ulysses James Joyce eBook use anyone anywhere cost al...|
|You may copy it, give it away or re-use it under the terms of the Project Gutenberg License inclu...|[[document, 0, 312, may copy give away reuse terms Project Gutenberg License inclu

In [99]:

#documentAssembler = DocumentAssembler()\
#.setInputCol("text")\
#.setOutputCol("document")

regex_matcher = RegexMatcher()\
    .setInputCols('sentences')\
    .setStrategy("MATCH_ALL")\
    .setOutputCol("regex_matches")\
    .setExternalRules(path='./ulyses_regex_rules.txt', delimiter=',')
    

nlpPipeline = Pipeline(stages=[
# documentAssembler, 
 regex_matcher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(cow_result)

#match_df.select('regex_matches.result').take(3)

In [100]:
match_df.select('regex_matches.result').show(50,truncate=100)

+-----------------------------------------------------------+
|                                                     result|
+-----------------------------------------------------------+
|                                                         []|
|                                                         []|
|                                                         []|
|[July, undoubtedly, July, 2,, 2014, #8387], 2005, 6,, 2003]|
|                                         [nearly, probably]|
|                                                         []|
|                                                         []|
|                                                         []|
|                                                         []|
|                                                         []|
|                                                   [highly]|
|                                     [slightly, invariably]|
|                                   [ruthlessly, fearlessly]|
|       

In [101]:
match_df.select('sentences.result','regex_matches.result')\
.toDF('sentences.result','matches').filter(F.size('matches')>=1)\
.show(truncate=100)

+----------------------------------------------------------------------------------------------------+-----------------------------------------------------------+
|                                                                                    sentences.result|                                                    matches|
+----------------------------------------------------------------------------------------------------+-----------------------------------------------------------+
|[Title: Hunger Author: Knut Hamsun Posting Date: October 2, 2014 [EBook #8387] Release Date: June...|[July, undoubtedly, July, 2,, 2014, #8387], 2005, 6,, 2003]|
|[Those approaching most nearly to his position are probably Selma Lagerlöf in Sweden and Henrik P...|                                         [nearly, probably]|
|[Instead, he must be classed as an individualistic romanticist and a highly subjective aristocrat...|                                                   [highly]|
|[He fears and flouts 

In [102]:
MODEL_NAME='classifierdl_use_cyberbullying'

In [103]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_cyberbullying download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


In [104]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(cow_result)


In [105]:
match_df.select('sentences.result').filter(match_df.sentiment.metadata['racism']>0.9).show(10,truncate=150)
                                                                              
#result_df = result.select(F.explode(F.arrays_zip('matched_entities.result', 'matched_entities.begin',  'matched_entities.end')).alias("cols1")) \
#.select(F.expr("cols1['0']").alias("matched_entities"),
#        F.expr("cols1['1']").alias("begin"),
#        F.expr("cols1['2']").alias("end"))#.toPandas()
#
#result_df.show(50)

AnalysisException: "cannot resolve '`sentiment`.`metadata`['racism']' due to data type mismatch: argument 2 requires integral type, however, ''racism'' is of string type.;;\n'Project [result#1634]\n+- 'Filter (sentiment#1617.metadata[racism] > 0.9)\n   +- Project [sentences#1192.result AS result#1634, sentiment#1617]\n      +- Project [text#1128, document#1566, sentences#1192, token#1197, normalized#1203, cleanTokens#1210, lemma#1218, clean_text#1227, token2#1237, pos#1248, chunk#1260, embeddings#1287, ner#1302, ner_chunk#1317, sentence_embeddings#1599, UDF(array(sentence_embeddings#1599)) AS sentiment#1617]\n         +- Project [text#1128, document#1566, sentences#1192, token#1197, normalized#1203, cleanTokens#1210, lemma#1218, clean_text#1227, token2#1237, pos#1248, chunk#1260, embeddings#1287, ner#1302, ner_chunk#1317, sentence_embeddings#1582 AS sentence_embeddings#1599]\n            +- Project [text#1128, document#1566, sentences#1192, token#1197, normalized#1203, cleanTokens#1210

In [106]:
match_df.filter(match_df['sentiment']['metadata']['racism'] > 0.9)

AnalysisException: "cannot resolve '`sentiment`.`metadata`['racism']' due to data type mismatch: argument 2 requires integral type, however, ''racism'' is of string type.;;\n'Filter (sentiment#1617.metadata[racism] > 0.9)\n+- Project [text#1128, document#1566, sentences#1192, token#1197, normalized#1203, cleanTokens#1210, lemma#1218, clean_text#1227, token2#1237, pos#1248, chunk#1260, embeddings#1287, ner#1302, ner_chunk#1317, sentence_embeddings#1599, UDF(array(sentence_embeddings#1599)) AS sentiment#1617]\n   +- Project [text#1128, document#1566, sentences#1192, token#1197, normalized#1203, cleanTokens#1210, lemma#1218, clean_text#1227, token2#1237, pos#1248, chunk#1260, embeddings#1287, ner#1302, ner_chunk#1317, sentence_embeddings#1582 AS sentence_embeddings#1599]\n      +- Project [text#1128, document#1566, sentences#1192, token#1197, normalized#1203, cleanTokens#1210, lemma#1218, clean_text#1227, token2#1237, pos#1248, chunk#1260, embeddings#1287, ner#1302, ner_chunk#1317, UDF(ar

In [708]:
match_df.select(F.explode('sentiment.metadata').alias('aytu')).select(F.explode("aytu")).show(10,truncate=270)

+--------+------------+
|     key|       value|
+--------+------------+
|sentence|           0|
|  sexism|1.0827814E-5|
| neutral|  0.99998486|
|  racism|  4.34277E-6|
|sentence|           0|
|  sexism| 4.161074E-6|
| neutral|    0.999995|
|  racism| 8.568252E-7|
|sentence|           0|
|  sexism| 0.035133682|
+--------+------------+
only showing top 10 rows



In [746]:
match_df.select('sentiment').take(10)

[Row(sentiment=[Row(annotatorType='category', begin=0, end=153, result='neutral', metadata={'sentence': '0', 'sexism': '1.0827814E-5', 'neutral': '0.99998486', 'racism': '4.34277E-6'}, embeddings=[])]),
 Row(sentiment=[Row(annotatorType='category', begin=0, end=399, result='neutral', metadata={'sentence': '0', 'sexism': '4.161074E-6', 'neutral': '0.999995', 'racism': '8.568252E-7'}, embeddings=[])]),
 Row(sentiment=[Row(annotatorType='category', begin=0, end=317, result='neutral', metadata={'sentence': '0', 'sexism': '0.035133682', 'neutral': '0.9636793', 'racism': '0.0011869894'}, embeddings=[])]),
 Row(sentiment=[Row(annotatorType='category', begin=0, end=89, result='neutral', metadata={'sentence': '0', 'sexism': '9.865905E-6', 'neutral': '0.99998915', 'racism': '9.0494876E-7'}, embeddings=[])]),
 Row(sentiment=[Row(annotatorType='category', begin=0, end=61, result='neutral', metadata={'sentence': '0', 'sexism': '1.6029236E-4', 'neutral': '0.9997644', 'racism': '7.531231E-5'}, embedd

In [686]:
a[0]['col']['racism']

'4.34277E-6'

In [108]:
match_df.filter(match_df['sentiment.result'][0] == 'sexism' ).select('document.result','sentiment.result').show(50,truncate=170)

+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|                                                                                                                                                                    result|  result|
+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------+--------+
|  [In the same way we know that the miraculous atmosphere of the Northland serves merely to develop and emphasize traits that lie slumbering in men and women everywhere.]|[sexism]|
|[It was a professedly psychological study that had about as much in common with the old-fashioned conceptions of man's mental activities as the delirious utterances of...|[sexism]|
|[Hamsun's ironical humor and whimsical manner of expression do more than the plot itself 

In [255]:

text = '<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>'

text_df = spark.createDataFrame([[text]]).toDF("text")


import re
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, IntegerType


#clean_text = lambda s: re.sub(r'<[^>]*>', '', s)
clean_text = lambda s: re.sub(r'\<.*?\>', '', s)

text_df.withColumn('cleaned', udf(clean_text, StringType())('text')).select('text','cleaned').show(truncate= False)

+----------------------------------------------------------------------------------------------+-----------------------+
|text                                                                                          |cleaned                |
+----------------------------------------------------------------------------------------------+-----------------------+
|<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>|Have a great birth day!|
+----------------------------------------------------------------------------------------------+-----------------------+



In [256]:

find_not_alnum_count = lambda s: len([i for i in s if not i.isalnum() and i!=' '])

find_not_alnum_count("it's your birth day!")

2

In [259]:
text_df.withColumn('cleaned', udf(find_not_alnum_count, IntegerType())('text')).select('text','cleaned').show(truncate= False)

+----------------------------------------------------------------------------------------------+-------+
|text                                                                                          |cleaned|
+----------------------------------------------------------------------------------------------+-------+
|<h1 style="color: #5e9ca0;">Have a great <span  style="color: #2b2301;">birth</span> day!</h1>|23     |
+----------------------------------------------------------------------------------------------+-------+



In [268]:
finisher = Finisher() \
    .setInputCols(["regex_matches"]) \
    .setIncludeMetadata(False) # set to False to remove metadata

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 regex_matcher,
 finisher
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

match_df = pipelineModel.transform(pubMedDF)

match_df.show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                            finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
|The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|                                                []|
|BACKGROUND: At present, it is one of the most i...|                                                []|
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Combined EEG/fMRI recording has been used to lo...|                                                []|
|Kohlschutter syndrome is a rare neurodegenerati...|                                                []|
|Statistical analysis of neuroimages is commonly...|                                                []|
|The synthetic DOX-LNA conjugate was characteriz...|            

In [261]:
match_df.printSchema()

root
 |-- text: string (nullable = true)
 |-- finished_regex_matches: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [262]:
match_df.filter(F.size('finished_regex_matches')>1).show(truncate = 50)

+--------------------------------------------------+--------------------------------------------------+
|                                              text|                            finished_regex_matches|
+--------------------------------------------------+--------------------------------------------------+
|OBJECTIVE: To investigate the relationship betw...|[renal failure, cardiac surgery, cardiac surger...|
|Aging is a physiological process that causes st...|                     [renal function, renal aging]|
|BACKGROUND: Left-sided inferior vena cava (IVC)...|          [renal surgery, renal mass, renal veins]|
|A successful kidney transplantation (Tx) offers...|                         [renal failure, renal Tx]|
|The transcription factor scleraxis has been imp...|[cardiac valves, cardiac fibroblasts, cardiac c...|
|Historically, the only surgical option for remo...|                        [renal mass, renal masses]|
|OBJECTIVES: To examine the clinical effectivene...|   [renal tr

In [271]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate("John and Peter are brothers. However they don't support each other that much.")

In [272]:
light_result.keys()

dict_keys(['document', 'token', 'stem', 'lemma'])

In [273]:
list(zip(light_result['token'], light_result['stem'], light_result['lemma']))

[('John', 'john', 'John'),
 ('and', 'and', 'and'),
 ('Peter', 'peter', 'Peter'),
 ('are', 'ar', 'be'),
 ('brothers', 'brother', 'brother'),
 ('.', '.', '.'),
 ('However', 'howev', 'However'),
 ('they', 'thei', 'they'),
 ("don't", "don't", "don't"),
 ('support', 'support', 'support'),
 ('each', 'each', 'each'),
 ('other', 'other', 'other'),
 ('that', 'that', 'that'),
 ('much', 'much', 'much'),
 ('.', '.', '.')]

In [285]:
pd.DataFrame([*zip(light_result['token'],light_result['lemma'],light_result['stem'])],columns=['token','lemma','stem'])

,token,lemma,stem
0,John,John,john
1,and,and,and
2,Peter,Peter,peter
3,are,be,ar
4,brothers,brother,brother
5,.,.,.
6,However,However,howev
7,they,they,thei
8,don't,don't,don't
9,support,support,support


In [286]:
text_list= ["How did serfdom develop in and then leave Russia ?",
"There will be some exciting breakthroughs in NLP this year."]

light_model.annotate(text_list)

[{'document': ['How did serfdom develop in and then leave Russia ?'],
  'token': ['How',
   'did',
   'serfdom',
   'develop',
   'in',
   'and',
   'then',
   'leave',
   'Russia',
   '?'],
  'stem': ['how',
   'did',
   'serfdom',
   'develop',
   'in',
   'and',
   'then',
   'leav',
   'russia',
   '?'],
  'lemma': ['How',
   'do',
   'serfdom',
   'develop',
   'in',
   'and',
   'then',
   'leave',
   'Russia',
   '?']},
 {'document': ['There will be some exciting breakthroughs in NLP this year.'],
  'token': ['There',
   'will',
   'be',
   'some',
   'exciting',
   'breakthroughs',
   'in',
   'NLP',
   'this',
   'year',
   '.'],
  'stem': ['there',
   'will',
   'be',
   'some',
   'excit',
   'breakthrough',
   'in',
   'nlp',
   'thi',
   'year',
   '.'],
  'lemma': ['There',
   'will',
   'be',
   'some',
   'exciting',
   'breakthrough',
   'in',
   'NLP',
   'this',
   'year',
   '.']}]

In [25]:
spark_df = spark.read.text("4300-0.txt").toDF('text')

In [26]:
spark_df.show(10)

+--------------------+
|                text|
+--------------------+
|                    |
|The Project Guten...|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|                    |
|      Title: Ulysses|
+--------------------+
only showing top 10 rows



In [27]:
spark_df=spark_df.filter(~spark_df.text.contains('['))\
            .filter(~spark_df.text.contains(']'))\
                .filter(~spark_df.text.rlike('\—\s*I+\s*\—'))\
                .filter(~spark_df.text.rlike('\*{3}\s*.*\s*\*{3}'))\

#result=result.filter(result1['text']!='')

In [28]:
spark_df.show(60)

+--------------------+
|                text|
+--------------------+
|                    |
|The Project Guten...|
|                    |
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|                    |
|                    |
|      Title: Ulysses|
|                    |
| Author: James Joyce|
|                    |
|Last Updated: Dec...|
|                    |
|   Language: English|
|                    |
|Character set enc...|
|                    |
|                    |
|                    |
|                    |
|                    |
|Produced by Col C...|
|                    |
|                    |
|                    |
|                    |
|                    |
|             Ulysses|
|                    |
|                    |
|      by James Joyce|
|                    |
|            Contents|
|                    |
|                    |
|                    |
|                    |
|                    |
|          

In [29]:
spark_df=spark_df.filter(spark_df['text']!='')

In [30]:
spark_df.show(20)

+--------------------+
|                text|
+--------------------+
|The Project Guten...|
|This eBook is for...|
|almost no restric...|
|re-use it under t...|
|with this eBook o...|
|      Title: Ulysses|
| Author: James Joyce|
|Last Updated: Dec...|
|   Language: English|
|Character set enc...|
|Produced by Col C...|
|             Ulysses|
|      by James Joyce|
|            Contents|
|Stately, plump Bu...|
|lather on which a...|
|dressinggown, ung...|
|morning air. He h...|
|—_Introibo ad alt...|
|Halted, he peered...|
+--------------------+
only showing top 20 rows



In [46]:
import pyspark.sql.functions as F

#news_df = spark.read\
#                .option("header", "true")\
#                .csv("news_category_test.csv")\
#                .withColumnRenamed("description", "text")
#
#news_df.show(truncate=50)

In [31]:
lemmatizer = LemmatizerModel.pretrained('lemma_antbnc', 'en') \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \


'''
lemmatizer = Lemmatizer() \
    .setInputCols(["token"]) \
    .setOutputCol("lemma") \
    .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")
'''

lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[OK!]


'\nlemmatizer = Lemmatizer()     .setInputCols(["token"])     .setOutputCol("lemma")     .setDictionary("./AntBNC_lemmas_ver_001.txt", value_delimiter ="\t", key_delimiter = "->")\n'

In [32]:
!cd ~/cache_pretrained && ls -l

total 12
drwxrwxr-x 4 aytu aytu 4096 Jun  7 22:27 explain_document_dl_en_2.4.3_2.4_1584626657780
drwxrwxr-x 4 aytu aytu 4096 Jun 11 22:11 lemma_antbnc_en_2.0.2_2.4_1556480454569
drwxrwxr-x 4 aytu aytu 4096 Jun  7 22:20 recognize_entities_dl_en_2.4.3_2.4_1584626752821


In [33]:
from pyspark.ml import Pipeline

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = Tokenizer() \
    .setInputCols(["document"]) \
    .setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)


In [34]:
result = pipelineModel.transform(spark_df)

result.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|The Project Guten...|[[document, 0, 53...|[[token, 0, 2, Th...|[[token, 0, 2, th...|[[token, 0, 2, Th...|
|This eBook is for...|[[document, 0, 63...|[[token, 0, 3, Th...|[[token, 0, 3, th...|[[token, 0, 3, Th...|
|almost no restric...|[[document, 0, 66...|[[token, 0, 5, al...|[[token, 0, 5, al...|[[token, 0, 5, al...|
|re-use it under t...|[[document, 0, 66...|[[token, 0, 5, re...|[[token, 0, 5, re...|[[token, 0, 5, re...|
|with this eBook o...|[[document, 0, 45...|[[token, 0, 3, wi...|[[token, 0, 3, wi...|[[token, 0, 3, wi...|
+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [35]:
result.select('token.result','lemma.result').show(5, truncate=100)

+------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+
|                                                                              result|                                                                             result|
+------------------------------------------------------------------------------------+-----------------------------------------------------------------------------------+
|                  [The, Project, Gutenberg, EBook, of, Ulysses, ,, by, James, Joyce]|                 [The, Project, Gutenberg, EBook, of, Ulysses, ,, by, James, Joyce]|
|     [This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with]|    [This, eBook, be, for, the, use, of, anyone, anywhere, at, no, cost, and, with]|
|[almost, no, restrictions, whatsoever, ., You, may, copy, it, ,, give, it, away, or]|[almost, no, restriction, whatsoever, ., You, may, copy, it

In [36]:
pos = PerceptronModel.pretrained("pos_anc", 'en')\
      .setInputCols("document", "token")\
      .setOutputCol("pos")

pos_anc download started this may take some time.
Approximate size to download 4.3 MB
[OK!]


In [37]:
!cd ~/cache_pretrained && ls -l

total 16
drwxrwxr-x 4 aytu aytu 4096 Jun  7 22:27 explain_document_dl_en_2.4.3_2.4_1584626657780
drwxrwxr-x 4 aytu aytu 4096 Jun 11 22:11 lemma_antbnc_en_2.0.2_2.4_1556480454569
drwxrwxr-x 4 aytu aytu 4096 Jun 11 22:13 pos_anc_en_2.0.2_2.4_1556659930154
drwxrwxr-x 4 aytu aytu 4096 Jun  7 22:20 recognize_entities_dl_en_2.4.3_2.4_1584626752821


In [38]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [40]:
result = pipelineModel.transform(spark_df)

result.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|                 pos|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|The Project Guten...|[[document, 0, 53...|[[token, 0, 2, Th...|[[token, 0, 2, th...|[[token, 0, 2, Th...|[[pos, 0, 2, DT, ...|
|This eBook is for...|[[document, 0, 63...|[[token, 0, 3, Th...|[[token, 0, 3, th...|[[token, 0, 3, Th...|[[pos, 0, 3, DT, ...|
|almost no restric...|[[document, 0, 66...|[[token, 0, 5, al...|[[token, 0, 5, al...|[[token, 0, 5, al...|[[pos, 0, 5, RB, ...|
|re-use it under t...|[[document, 0, 66...|[[token, 0, 5, re...|[[token, 0, 5, re...|[[token, 0, 5, re...|[[pos, 0, 5, NN, ...|
|with this eBook o...|[[document, 0, 45...|[[token, 0, 3, wi...|[[token, 0, 3, wi...|[[token, 0, 3, wi..

In [42]:
result.select('token.result','pos.result').show(25, truncate=100)

+----------------------------------------------------------------------------------------+-------------------------------------------------------------+
|                                                                                  result|                                                       result|
+----------------------------------------------------------------------------------------+-------------------------------------------------------------+
|                      [The, Project, Gutenberg, EBook, of, Ulysses, ,, by, James, Joyce]|                [DT, NNP, NNP, NNP, IN, NNP, ,, IN, NNP, NNP]|
|         [This, eBook, is, for, the, use, of, anyone, anywhere, at, no, cost, and, with]|    [DT, NN, VBZ, IN, DT, NN, IN, NN, RB, IN, DT, NN, CC, IN]|
|    [almost, no, restrictions, whatsoever, ., You, may, copy, it, ,, give, it, away, or]|   [RB, DT, NNS, RB, ., PRP, MD, VB, PRP, ,, VB, PRP, RB, CC]|
|         [re-use, it, under, the, terms, of, the, Project, Gutenberg, License, in

In [47]:
result = pipelineModel.transform(spark_df.limit(100))

result_df = result.select(F.explode(F.arrays_zip('token.result', 'token.begin', 'token.end', 'stem.result',  'lemma.result', 'pos.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']").alias("stem"),
        F.expr("cols['4']").alias("lemma"),
        F.expr("cols['5']").alias("pos")).toPandas()

result_df.head(10) 

,token,begin,end,stem,lemma,pos
0,The,0,2,the,The,DT
1,Project,4,10,project,Project,NNP
2,Gutenberg,12,20,gutenberg,Gutenberg,NNP
3,EBook,22,26,ebook,EBook,NNP
4,of,28,29,of,of,IN
5,Ulysses,31,37,ulyss,Ulysses,NNP
6,",",38,38,",",",",","
7,by,40,41,by,by,IN
8,James,43,47,jame,James,NNP
9,Joyce,49,53,joyc,Joyce,NNP


In [48]:
# same in LightPipeline

light_model = LightPipeline(pipelineModel)

light_result = light_model.annotate('Unions representing workers at Turner   Newall say they are disappointed after talks with stricken parent firm Federal Mogul.')

list(zip(light_result['token'], light_result['stem'], light_result['lemma'], light_result['pos']))

[('Unions', 'union', 'Unions', 'NNP'),
 ('representing', 'repres', 'represent', 'VBG'),
 ('workers', 'worker', 'worker', 'NNS'),
 ('at', 'at', 'at', 'IN'),
 ('Turner', 'turner', 'Turner', 'NNP'),
 ('Newall', 'newal', 'Newall', 'NNP'),
 ('say', 'sai', 'say', 'VBP'),
 ('they', 'thei', 'they', 'PRP'),
 ('are', 'ar', 'be', 'VBP'),
 ('disappointed', 'disappoint', 'disappointed', 'VBN'),
 ('after', 'after', 'after', 'IN'),
 ('talks', 'talk', 'talk', 'NNS'),
 ('with', 'with', 'with', 'IN'),
 ('stricken', 'stricken', 'stricken', 'NN'),
 ('parent', 'parent', 'parent', 'NN'),
 ('firm', 'firm', 'firm', 'NN'),
 ('Federal', 'feder', 'Federal', 'NNP'),
 ('Mogul', 'mogul', 'Mogul', 'NNP'),
 ('.', '.', '.', '.')]

In [50]:
light_result.keys()

dict_keys(['stem', 'lemma', 'document', 'pos', 'token'])

In [101]:
# applying POS chunker to find a custom pattern

chunker = Chunker()\
    .setInputCols(["document", "pos"])\
    .setOutputCol("chunk")\
    .setRegexParsers(["<DT>+<JJ>*<NN>"])


# NNP: Proper Noun
# NN: COmmon Noun
# DT: Determinator (e.g. the)
# JJ: Adjective

nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos,
 chunker
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [102]:
result = pipelineModel.transform(spark_df.limit(100))

result.show(5)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|               token|                stem|               lemma|                 pos|               chunk|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|The Project Guten...|[[document, 0, 53...|[[token, 0, 2, Th...|[[token, 0, 2, th...|[[token, 0, 2, Th...|[[pos, 0, 2, DT, ...|                  []|
|This eBook is for...|[[document, 0, 63...|[[token, 0, 3, Th...|[[token, 0, 3, th...|[[token, 0, 3, Th...|[[pos, 0, 3, DT, ...|[[chunk, 0, 9, Th...|
|almost no restric...|[[document, 0, 66...|[[token, 0, 5, al...|[[token, 0, 5, al...|[[token, 0, 5, al...|[[pos, 0, 5, RB, ...|                  []|
|re-use it under t...|[[document, 0, 66...|[[token, 0, 5, re...|[[token, 0, 5, re...|[[token, 0, 5, re...|

In [104]:
result.select('chunk').take(20)

[Row(chunk=[]),
 Row(chunk=[Row(annotatorType='chunk', begin=0, end=9, result='This eBook', metadata={'sentence': '0', 'chunk': '0'}, embeddings=[]), Row(annotatorType='chunk', begin=18, end=24, result='the use', metadata={'sentence': '0', 'chunk': '1'}, embeddings=[]), Row(annotatorType='chunk', begin=48, end=54, result='no cost', metadata={'sentence': '0', 'chunk': '2'}, embeddings=[])]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[Row(annotatorType='chunk', begin=5, end=14, result='this eBook', metadata={'sentence': '0', 'chunk': '0'}, embeddings=[])]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[]),
 Row(chunk=[Row(annotatorType='chunk', begin=39, end=51, result='the stairhead', metadata={'sentence': '0', 'chunk': '0'}, embeddings=[]), Row(annotatorType='chunk', begin=62, end=67, result='a bowl', metadata={'sentence': '0', 'chunk': '1'}, embeddings=[])]),
 Row(chunk=[Row(annotatorType='c

In [105]:
result_df = result.select(F.explode(F.arrays_zip('chunk.result', 'chunk.begin',  'chunk.end')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end")).toPandas()

result_df.head(10)

,chunk,begin,end
0,This eBook,0,9
1,the use,18,24
2,no cost,48,54
3,this eBook,5,14
4,the stairhead,39,51
5,a bowl,62,67
6,a mirror,16,23
7,a razor,29,35
8,A yellow,50,57
9,the mild,60,67


In [106]:
dep_parser = DependencyParserModel.pretrained('dependency_conllu')\
        .setInputCols(["document", "pos", "token"])\
        .setOutputCol("dependency")

dependency_conllu download started this may take some time.
Approximate size to download 16.6 MB
[OK!]


In [107]:
typed_dep_parser = TypedDependencyParserModel.pretrained('dependency_typed_conllu')\
        .setInputCols(["token", "pos", "dependency"])\
        .setOutputCol("dependency_type")

dependency_typed_conllu download started this may take some time.
Approximate size to download 257.4 KB
[OK!]


In [108]:
nlpPipeline = Pipeline(stages=[
 documentAssembler, 
 tokenizer,
 stemmer,
 lemmatizer,
 pos,
 dep_parser,
 typed_dep_parser
 ])

empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)

In [111]:
sdf=sp

result = pipelineModel.transform("The sun is shining and i will go out for biking.")

result_df = result.select(F.explode(F.arrays_zip('token.result', 'token.begin',  'token.end', 'dependency.result', 'dependency_type.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']").alias("dependency"),
        F.expr("cols['4']").alias("dependency_type")).toPandas()

result_df.head(10)

AttributeError: 'str' object has no attribute '_jdf'

In [324]:
full=doc_df.select("document.result").take(1)[0][0]

In [327]:
chars=list(set(names))

In [333]:
full=full[0]

In [381]:
sorted(chars)

[' Church',
 ' Clapham',
 ' Dame',
 ' Talbot',
 'A Pisgah',
 'Aaron Figatner',
 'Abe Kirschner',
 'Abraham Chatterton',
 'Adam Kadmon',
 'Advocate Bushe',
 'Ae Arval',
 'Agendath Netaim',
 'Agnus Dei',
 'Aitcha Ha',
 'Alba Longa',
 'Albert Edward',
 'Alderman Cowley',
 'Alexander Keyes',
 'Alf Bergan',
 'Alias Ledwidge',
 'Allwombing Tomb',
 'Ally Sloper',
 'Almidano Artifoni',
 'Aloysius Gonzaga',
 'Anna Liffey',
 'Anne Kearns',
 'Anthony Macdonnell',
 'Ape Swillale',
 'Arabella Blackwood',
 'Arbour Hill',
 'Argive Helen',
 'Arnold Dolmetsch',
 'Arthur Chace',
 'Arthur Edmund',
 'Arthur Griffith',
 'Avvocato Pagamimi',
 'Baby Boardman',
 'Bacibaci Beninobenone',
 'Baddybad Stephen',
 'Bags Comisky',
 'Bald Pat',
 'Ballocky Mulligan',
 'Bandmann Palmer',
 'Bantam Lyons',
 'Barbara Lovebirch',
 'Barney Kiernan',
 'Baron Leopold',
 'Bartell Darcy',
 'Barton James',
 'Bathing Crissie',
 'Beau Mount',
 'Beaufoy Purefoy',
 'Bella Cohen',
 'Ben Dollard',
 'Ben Howth',
 'Ben Maimun',
 'Benjam

In [334]:
re.findall('Monsieur Moore',full,flags=re.I)

['Monsieur Moore', 'Monsieur Moore']

In [337]:
[re.findall("^.*"+Cardinal".*$",full,flags=re.I) for char in chars if re.findall(char,full,flags=re.I)!=[]]

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],


In [346]:
regex="\A"+chars[0]+"\Z"

In [347]:
re.findall(regex,sentences,flags=re.I)

[]

In [349]:
sentences

NameError: name 'sentences' is not defined

In [357]:
df=sent_df.select(F.explode('sentences.result').alias('text')).toPandas()

In [363]:
sentences=df.text.to_list()

In [370]:
from tqdm import tqdm_notebook as tqdm

In [376]:
[[(char,sentence) for  sentence in (sentences) if re.findall(char,full,flags=re.I)!=[]] for char in tqdm(chars)]

/home/aytu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


KeyboardInterrupt: 

In [379]:
res = []

for char in tqdm(chars):
    
    for  sentence in (sentences):
        
        if re.findall(char,sentence,flags=re.I)!=[]:
            
            res.append((char,sentence))
            
            break


#[[(char,sentence) for  sentence in (sentences) if re.findall(char,full,flags=re.I)!=[]] for char in tqdm(chars)]

/home/aytu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  This is separate from the ipykernel package so we can avoid doing imports until


In [266]:
spark_df1 = spark.read.csv('Tweets.csv',header=True)
spark_df1 = spark_df1.select('text','airline_sentiment').filter("text is not null")

In [267]:
spark_df1.columns

['text', 'airline_sentiment']

In [254]:
spark_df1 =  spark_df1.select(F.regexp_replace('text',"\w+:\/{2}[\d\w\-]+(\.[\d\w\-]+)*(?:(?:\/[^\s/]*))*","").alias('document'),'sentiment').filter("document is not null")\
                .select(F.regexp_replace('document',"http.{1,10}…","").alias('document'),'sentiment').select(F.regexp_replace('document',"…","").alias('document'),'sentiment')

AnalysisException: "cannot resolve '`sentiment`' given input columns: [text, airline_sentiment];;\n'Project [regexp_replace(text#2868, \\w+:\\/{2}[\\d\\w\\-]+(\\.[\\d\\w\\-]+)*(?:(?:\\/[^\\s/]*))*, ) AS document#2890, 'sentiment]\n+- Filter isnotnull(text#2868)\n   +- Project [text#2868, airline_sentiment#2859]\n      +- Relation[tweet_id#2858,airline_sentiment#2859,airline_sentiment_confidence#2860,negativereason#2861,negativereason_confidence#2862,airline#2863,airline_sentiment_gold#2864,name#2865,negativereason_gold#2866,retweet_count#2867,text#2868,tweet_coord#2869,tweet_created#2870,tweet_location#2871,user_timezone#2872] csv\n"

In [256]:
spark_df1.show(10,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------+-----------------+
|text                                                                                                                              |airline_sentiment|
+----------------------------------------------------------------------------------------------------------------------------------+-----------------+
|@VirginAmerica What @dhepburn said.                                                                                               |neutral          |
|@VirginAmerica plus you've added commercials to the experience... tacky.                                                          |positive         |
|@VirginAmerica I didn't today... Must mean I need to take another trip!                                                           |neutral          |
|"@VirginAmerica it's really aggressive to blast obnoxious ""entertainment"" in your guests' f

In [265]:
spark_df1 = spark_df1.select(F.regexp_replace('text',"RT\s.*\:\s","").alias('document'),'sentiment').filter("document is not null")
                


AnalysisException: "cannot resolve '`text`' given input columns: [document, airline_sentiment];;\n'Project [regexp_replace('text, RT\\s.*\\:\\s, ) AS regexp_replace(text, RT\\s.*\\:\\s, )#2917, 'sentiment]\n+- Filter isnotnull(document#2901)\n   +- Project [regexp_replace(text#2868, RT\\s.*\\:\\s, ) AS document#2901, airline_sentiment#2859]\n      +- Filter isnotnull(text#2868)\n         +- Project [text#2868, airline_sentiment#2859]\n            +- Relation[tweet_id#2858,airline_sentiment#2859,airline_sentiment_confidence#2860,negativereason#2861,negativereason_confidence#2862,airline#2863,airline_sentiment_gold#2864,name#2865,negativereason_gold#2866,retweet_count#2867,text#2868,tweet_coord#2869,tweet_created#2870,tweet_location#2871,user_timezone#2872] csv\n"

In [268]:
spark_df1.show()

+--------------------+-----------------+
|                text|airline_sentiment|
+--------------------+-----------------+
|@VirginAmerica Wh...|          neutral|
|@VirginAmerica pl...|         positive|
|@VirginAmerica I ...|          neutral|
|"@VirginAmerica i...|         negative|
|@VirginAmerica an...|         negative|
|@VirginAmerica se...|         negative|
|@VirginAmerica ye...|         positive|
|@VirginAmerica Re...|          neutral|
|@virginamerica We...|         positive|
|@VirginAmerica it...|         positive|
|@VirginAmerica di...|          neutral|
|@VirginAmerica I ...|         positive|
|@VirginAmerica Th...|         positive|
|@VirginAmerica @v...|         positive|
|@VirginAmerica Th...|         positive|
|@VirginAmerica SF...|         negative|
|@VirginAmerica So...|         positive|
|@VirginAmerica  I...|         negative|
|I ❤️ flying @Virg...|         positive|
|@VirginAmerica yo...|         positive|
+--------------------+-----------------+
only showing top

In [259]:
import json
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [269]:
MODEL_NAME='sentimentdl_use_twitter'

In [270]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")



use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = SentimentDLModel.pretrained(name=MODEL_NAME, lang="en")\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("pr_sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.9 MB
[OK!]


In [271]:
empty_df = spark.createDataFrame([['']]).toDF("text")

pipelineModel = nlpPipeline.fit(empty_df)



In [272]:
#df = spark.createDataFrame(pd.DataFrame({"text":text_list}))
result3 = pipelineModel.transform(spark_df1)

In [277]:
result3=result3.withColumnRenamed('airline_sentiment','sentiment')

In [278]:
result3.show()

+--------------------+---------+--------------------+--------------------+--------------------+
|                text|sentiment|            document| sentence_embeddings|        pr_sentiment|
+--------------------+---------+--------------------+--------------------+--------------------+
|@VirginAmerica Wh...|  neutral|[[document, 0, 34...|[[sentence_embedd...|[[category, 0, 34...|
|@VirginAmerica pl...| positive|[[document, 0, 71...|[[sentence_embedd...|[[category, 0, 71...|
|@VirginAmerica I ...|  neutral|[[document, 0, 70...|[[sentence_embedd...|[[category, 0, 70...|
|"@VirginAmerica i...| negative|[[document, 0, 12...|[[sentence_embedd...|[[category, 0, 12...|
|@VirginAmerica an...| negative|[[document, 0, 54...|[[sentence_embedd...|[[category, 0, 54...|
|@VirginAmerica se...| negative|[[document, 0, 87...|[[sentence_embedd...|[[category, 0, 87...|
|@VirginAmerica ye...| positive|[[document, 0, 78...|[[sentence_embedd...|[[category, 0, 78...|
|@VirginAmerica Re...|  neutral|[[docume

In [279]:
result4=result3.select(F.explode(F.arrays_zip('document.result', 'pr_sentiment.result')).alias("cols"),'sentiment').filter("pr_sentiment is not null").filter("document.result is not null").filter("pr_sentiment.result is not null") \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("pr_sentiment"),'sentiment')

In [296]:
result4.show(100,truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+---------+
|document                                                                                                                                                     |pr_sentiment|sentiment|
+-------------------------------------------------------------------------------------------------------------------------------------------------------------+------------+---------+
|@VirginAmerica What @dhepburn said.                                                                                                                          |positive    |neutral  |
|@VirginAmerica plus you've added commercials to the experience... tacky.                                                                                     |positive    |positive |
|@VirginAmerica I didn't today... Must mean I need to take another trip!             

In [281]:
result4=result4.withColumn('sentiment', F.lower(F.col('sentiment')))

In [297]:
result4.filter(result4["pr_sentiment"]==result4["sentiment"]).count()

9560

In [298]:
result4.filter(result4["pr_sentiment"]!=result4["sentiment"]).count()

5072

In [299]:
9560/(9560+5072)

0.6533624931656643

In [213]:
result4 = result4.withColumn('same',F.col("pr_sentiment")==F.col("sentiment"))

In [229]:
result4.filter(F.col('same')==1).select(F.count('same')).collect()

[Row(count(same)=5939)]

In [230]:
result4.filter(F.col('same')==0).select(F.count('same')).collect()

[Row(count(same)=7926)]

In [242]:
MODEL_NAME='classifierdl_use_cyberbullying'

In [243]:
documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en")\
 .setInputCols(["document"])\
 .setOutputCol("sentence_embeddings")


sentimentdl = ClassifierDLModel.pretrained(name=MODEL_NAME)\
    .setInputCols(["sentence_embeddings"])\
    .setOutputCol("sentiment")

nlpPipeline = Pipeline(
      stages = [
          documentAssembler,
          use,
          sentimentdl
      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
classifierdl_use_cyberbullying download started this may take some time.
Approximate size to download 21.4 MB
[OK!]


In [244]:
pipelineModel = nlpPipeline.fit(empty_df)
result3 = pipelineModel.transform(spark_df1)

In [245]:
result3.select(F.explode(F.arrays_zip('document.result', 'sentiment.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("document"),
        F.expr("cols['1']").alias("sentiment")).show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------------------------------+---------+
|document                                                                                                                                    |sentiment|
+--------------------------------------------------------------------------------------------------------------------------------------------+---------+
|RT @NancyLeeGrahn: How did everyone feel about the Climate Change question last night? Exactly. #GOPDebate                                  |neutral  |
|RT @ScottWalker: Didn't catch the full #GOPdebate last night. Here are some of Scott's best lines in 90 seconds. #Walker16 http://t.co/ZSfF…|neutral  |
|RT @TJMShow: No mention of Tamir Rice and the #GOPDebate was held in Cleveland? Wow.                                                        |neutral  |
|RT @RobGeorge: That Carly Fiorina is trending -- hours after HER debate -- above 

In [236]:
result4.show()

+--------------------+------------+---------+
|            document|pr_sentiment|sentiment|
+--------------------+------------+---------+
|RT @NancyLeeGrahn...|    positive|  Neutral|
|RT @ScottWalker: ...|    positive| Positive|
|RT @TJMShow: No m...|    negative|  Neutral|
|RT @RobGeorge: Th...|    negative| Positive|
|RT @DanScavino: #...|    positive| Positive|
|"RT @GregAbbott_T...|    negative| Positive|
|RT @warriorwoman9...|     neutral| Negative|
|Going on #MSNBC L...|    positive|  Neutral|
|Deer in the headl...|    negative| Negative|
|RT @NancyOsborne1...|    positive| Negative|
|@JGreenDC @realDo...|    negative| Negative|
|RT @WayneDupreeSh...|    positive| Positive|
|Me reading my fam...|    positive| Negative|
|"RT @ArcticFox201...|    positive|  Neutral|
|RT @pattonoswalt:...|    positive| Positive|
|Hey @ChrisChristi...|    negative| Negative|
|RT @CarolCNN: #Do...|    negative| Negative|
|RT @johncardillo:...|    positive| Negative|
|reason comment is...|    negative